In [1]:
import pandas as pd

In [2]:
# Fonte dos dados
# https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2018.zip
# https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2022.zip

In [3]:
dataset_2018 = r'C:\Users\rgarcia\Documents\data\consulta_cand_2018_BRASIL.csv'
dataset_2022 = r'C:\Users\rgarcia\Documents\data\consulta_cand_2022_BRASIL.csv'

In [4]:
cand_2018 = pd.read_csv(dataset_2018, sep=';', encoding='ISO-8859-1', decimal=',', low_memory=False)
cand_2022 = pd.read_csv(dataset_2022, sep=';', encoding='ISO-8859-1', decimal=',', low_memory=False)

In [5]:
cand_2018.DS_SITUACAO_CANDIDATO_PLEITO.unique()

array(['DEFERIDO', '#NULO#', 'RENÚNCIA', 'PEDIDO NÃO CONHECIDO',
       'INDEFERIDO', 'DEFERIDO COM RECURSO', 'INDEFERIDO COM RECURSO',
       'PENDENTE DE JULGAMENTO', 'AGUARDANDO JULGAMENTO', 'FALECIDO'],
      dtype=object)

In [6]:
filtra_deferidos = False

In [7]:
if filtra_deferidos:
    cand_2018 = cand_2018[cand_2018.DS_SITUACAO_CANDIDATO_PLEITO.isin(['DEFERIDO', 'DEFERIDO COM RECURSO'])]

## Histórico

In [9]:
historico = pd.DataFrame({'Eleição':[1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022],
             'Candidatos':[6623, 15040, 18049, 19263, 22537, 26161, 29085, cand_2022.shape[0]]})

historico['Variação (%)'] = (historico['Candidatos'].pct_change()*100).round(1)

#### Eleição 2022 deve apresentar primeira<br>queda no número total de candidatos

In [10]:
historico

,Eleição,Candidatos,Variação (%)
0,1994,6623,NaN
1,1998,15040,127.1
2,2002,18049,20.0
3,2006,19263,6.7
4,2010,22537,17.0
5,2014,26161,16.1
6,2018,29085,11.2
7,2022,28817,-0.9


In [45]:
# cand_2018.columns

## Cenário por cargo disputado

In [13]:
cand_cargo_2018 = cand_2018.groupby(['DS_CARGO']).SQ_CANDIDATO.count().reset_index()

In [14]:
cand_cargo_2022 = cand_2022.groupby(['DS_CARGO']).SQ_CANDIDATO.count().reset_index()

In [15]:
cand_cargo_2022['ANO_ELEICAO'] = 2022

In [16]:
cand_cargo_2018['ANO_ELEICAO'] = 2018

In [17]:
df = pd.concat([cand_cargo_2018, cand_cargo_2022]).reset_index(drop=True)

In [18]:
df = df.pivot(columns='ANO_ELEICAO', index='DS_CARGO', values='SQ_CANDIDATO').reset_index()

In [19]:
df['Variação'] = ((df[2022]/df[2018]-1)*100).round(1)

In [20]:
df = df.reset_index(drop=True)

In [21]:
df.index.names = ['']

In [22]:
df.columns.name = ''

In [23]:
df.columns = ['Cargo disputado', '2018', '2022', 'Variação (%)']

In [24]:
df['Cargo disputado'] = df['Cargo disputado'].str.title()

In [25]:
df['Direção'] = df['Variação (%)'].apply(lambda x: 'Aumento' if x > 0 else 'Queda')

#### Queda ocorreu em postulantes a todos<br>os cargos, menos deputado federal

In [26]:
df

,Cargo disputado,2018,2022,Variação (%),Direção
,,,,,
0,1º Suplente,398,245,-38.4,Queda
1,2º Suplente,405,249,-38.5,Queda
2,Deputado Distrital,981,598,-39.0,Queda
3,Deputado Estadual,17941,16530,-7.9,Queda
4,Deputado Federal,8588,10482,22.1,Aumento
5,Governador,230,223,-3.0,Queda
6,Presidente,16,12,-25.0,Queda
7,Senador,369,237,-35.8,Queda
8,Vice-Governador,236,229,-3.0,Queda


## Deputado estadual por UF

In [27]:
dep_est_2018 = cand_2018[cand_2018.DS_CARGO == 'DEPUTADO ESTADUAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_2022 = cand_2022[cand_2022.DS_CARGO == 'DEPUTADO ESTADUAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_uf = pd.concat([dep_est_2018, dep_est_2022]).pivot(index='SG_UF', columns='ANO_ELEICAO', values= 'SQ_CANDIDATO').reset_index()

dep_est_uf.columns.names = ['']

dep_est_uf['Variação (%)'] = ((dep_est_uf[2022]/dep_est_uf[2018]-1)*100).round(1)

dep_est_uf.columns.names = ['Deputado estadual']

dep_est_uf = dep_est_uf.sort_values(by='Variação (%)', ascending=False).reset_index(drop=True)

#### Número de deputados estauduais<br>no RJ é o que mais puxa a queda

In [28]:
dep_est_uf

Deputado estadual,SG_UF,2018,2022,Variação (%)
0,BA,643,902,40.3
1,TO,234,310,32.5
2,SC,469,610,30.1
3,PR,767,894,16.6
4,MS,355,397,11.8
5,PB,429,464,8.2
6,MA,535,549,2.6
7,MG,1382,1399,1.2
8,RS,853,825,-3.3
9,RN,330,318,-3.6


## Deputado federal por UF

In [29]:
dep_fed_2018 = cand_2018[cand_2018.DS_CARGO == 'DEPUTADO FEDERAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_2022 = cand_2022[cand_2022.DS_CARGO == 'DEPUTADO FEDERAL'].groupby(['SG_UF', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_uf = pd.concat([dep_fed_2018, dep_fed_2022]).pivot(index='SG_UF', columns='ANO_ELEICAO', values= 'SQ_CANDIDATO').reset_index()

dep_fed_uf['Variação (%)'] = ((dep_fed_uf[2022]/dep_fed_uf[2018]-1)*100).round(1)

dep_fed_uf.columns.names = ['Deputado Federal']

dep_fed_uf = dep_fed_uf.sort_values(by='Variação (%)', ascending=False).reset_index(drop=True)



#### Entre deputados federais, houve pequena queda em SP<br>e RJ, mas Norte e Nordeste tiveram grande aumento

In [30]:
dep_fed_uf

Deputado Federal,SG_UF,2018,2022,Variação (%)
0,PA,141,314,122.7
1,AL,83,181,118.1
2,TO,88,159,80.7
3,MA,213,360,69.0
4,GO,229,386,68.6
5,AC,84,138,64.3
6,CE,265,410,54.7
7,PB,159,242,52.2
8,RN,121,184,52.1
9,BA,503,763,51.7


In [31]:
nome_partido = {'PR':'PL', 'PPS':'CIDADANIA', 'PRB':'REPUBLICANOS', 'DEM':'UNIÃO (DEM+PSL)', 'PTC':'AGIR', 'PSL':'UNIÃO (DEM+PSL)',
                'PPL': 'PC do B', 'PRP':'PATRIOTA', 'PHS':'PODE'}

In [32]:
cand_2018.SG_PARTIDO = cand_2018.SG_PARTIDO.replace(nome_partido)

In [33]:
cand_2022.SG_PARTIDO = cand_2022.SG_PARTIDO.replace('UNIÃO','UNIÃO (DEM+PSL)')

In [34]:
dep_est_partido_2022 = cand_2022[cand_2022['DS_CARGO'] == 'DEPUTADO ESTADUAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_partido_2018 = cand_2018[cand_2018['DS_CARGO'] == 'DEPUTADO ESTADUAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_est_compara = pd.concat([dep_est_partido_2018, 
           dep_est_partido_2022]).pivot(columns='ANO_ELEICAO', 
                                                              index='SG_PARTIDO', values='SQ_CANDIDATO').fillna(0).astype(int)

dep_est_compara['Variação (%)'] = (((dep_est_compara[2022]/dep_est_compara[2018])-1)*100).round(1)

dep_est_compara = dep_est_compara.sort_values(by='Variação (%)', ascending=False)

In [35]:
dep_fed_partido_2022 = cand_2022[cand_2022['DS_CARGO'] == 'DEPUTADO FEDERAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_partido_2018 = cand_2018[cand_2018['DS_CARGO'] == 'DEPUTADO FEDERAL'].groupby(['SG_PARTIDO', 'ANO_ELEICAO']).SQ_CANDIDATO.count().reset_index()

dep_fed_compara = pd.concat([dep_fed_partido_2018, 
           dep_fed_partido_2022]).pivot(columns='ANO_ELEICAO', 
                                                              index='SG_PARTIDO', values='SQ_CANDIDATO').fillna(0).astype(int)

dep_fed_compara['Variação (%)'] = (((dep_fed_compara[2022]/dep_fed_compara[2018])-1)*100).round(1)

dep_fed_compara = dep_fed_compara.sort_values(by='Variação (%)', ascending=False)

In [36]:
dep_est_compara.columns.name = "Deputado Estadual"

In [37]:
dep_fed_compara.columns.name = "Deputado Federal"

#### A quantidade de candidatos a deputado federal por partido<br>sugere que a redução se deveu principalmente ao<br> encolhimento de partidos de esquerda. A exceção foi o<br> encolhimento de PSL e DEM (agora União) esvaziado pela<br> ida de Bolsonaro ao PL

In [38]:
dep_est_compara

Deputado Estadual,2018,2022,Variação (%)
SG_PARTIDO,,,
UP,0,19,inf
PL,472,990,109.7
PMB,297,557,87.5
PTB,424,760,79.2
NOVO,125,212,69.6
REPUBLICANOS,533,866,62.5
PSD,435,682,56.8
PP,524,775,47.9
PDT,572,784,37.1


#### Entre deputados federais, o inchaço da direita<br> também é perceptível, ainda que a esquerda<br> não tenha encolhido tanto

In [44]:
dep_fed_compara

Deputado Federal,SG_PARTIDO,2018,2022,Variação (%)
0,UP,0,17,inf
1,PTB,144,497,245.1
2,PP,151,508,236.4
3,PL,164,505,207.9
4,SOLIDARIEDADE,163,407,149.7
5,PSC,189,421,122.8
6,PSD,208,427,105.3
7,REPUBLICANOS,259,523,101.9
8,PMB,117,227,94.0
9,PCO,31,56,80.6


In [41]:
dep_fed_compara = dep_fed_compara.reset_index()

In [42]:
dep_est_compara = dep_est_compara.reset_index()

In [105]:
piramide_etaria = cand_2022.groupby([
    'NR_IDADE_DATA_POSSE', 
    'DS_GENERO']).SQ_CANDIDATO.count().reset_index()

In [113]:
age_group = lambda x: \
'18 a 19' if x < 20 else \
'20 a 24' if x < 25 else \
'25 a 29' if x < 30 else \
'30 a 34' if x < 35 else \
'35 a 39' if x < 40 else \
'40 a 44' if x < 45 else \
'45 a 49' if x < 50 else \
'50 a 54' if x < 55 else \
'55 a 59' if x < 60 else \
'60 a 64' if x < 65 else \
'65 a 69' if x < 70 else \
'70 a 74' if x < 75 else \
'75 a 79' if x < 80 else \
'80 ou mais'

In [108]:
piramide_etaria['Faixa'] = piramide_etaria.NR_IDADE_DATA_POSSE.apply(age_group)

In [109]:
piramide_etaria = piramide_etaria.groupby(['Faixa', 'DS_GENERO']).SQ_CANDIDATO.sum().reset_index().pivot(index='Faixa', columns='DS_GENERO', values='SQ_CANDIDATO').fillna(0).astype(int).reset_index()

In [110]:
piramide_etaria.FEMININO = piramide_etaria.FEMININO*-1

In [111]:
piramide_etaria.to_csv(f'piramide_etaria.csv', index=False)

In [114]:
count = 1

for i in [historico, df, dep_est_uf, dep_fed_uf, dep_est_compara, dep_fed_compara, piramide_etaria]:

    i.to_csv(f'tabela_{count}.csv', index=False)
    count += 1